In [266]:
import sys
from yahoo_fin import stock_info as si
import numpy as np
import pandas as pd
import time
import os
from termcolor import colored

cache_dir = 'cache_dir'
if not os.path.isdir(cache_dir):
    os.mkdir(cache_dir)

In [267]:
def get_new_ticker(ticker_, interval):
    new_ticker = pd.DataFrame
    file_ticker = os.path.join(cache_dir,ticker_ + '_' + time.strftime("%Y-%m-%d") + '_' + interval + ".csv")
    if not os.path.isfile(file_ticker):
        new_ticker = si.get_data(ticker_, interval=interval)
        if "date" not in new_ticker.columns:
            new_ticker["date"] = new_ticker.index
        new_ticker.to_csv(file_ticker, index=False)
    if os.path.isfile(file_ticker):
        new_ticker = pd.read_csv(file_ticker,index_col='date')
        new_ticker.rename(columns={'close':ticker_}, inplace=True)
        new_ticker.drop(['open', 'high','low','volume','ticker','adjclose'], axis='columns', inplace=True)
    return new_ticker

def portfolio(tickers_list, interval):
    df_ticker = pd.DataFrame
    for ticker in tickers_list:
        if not df_ticker.empty:
            df_ticker = pd.concat([df_ticker, get_new_ticker(ticker, interval) ], axis=1)
        else:
            df_ticker = pd.concat([get_new_ticker(ticker, interval) ], axis=1)
    return df_ticker

In [268]:
tickers = [
    'BTC-USD',
    # 'VIXY',
    '^VIX',
    'VOO', #Vanguard S&P 500 ETF
    'QQQ',
    'KWEB',
    'VNQ',
    'TLT', #iShares 20+ Year Treasury Bond ETF (TLT)
    'BAR',
    # # 'GLD',
    'COMT', # iShares GSCI Commodity Dynamic Roll Strategy ETF (COMT)
    # # 'DBC', # Invesco DB Commodity Index Tracking Fund (DBC)
]
# 14%	14%	5%      22% 13% 10% 10%     5%      7%
# VOO   QQQ KWEB    VNQ TLT BAR COMT    VIXY    BTC-USD
df = portfolio(tickers,'1mo') #“1d”, “1wk”, “1mo”
df.tail(5)
df.to_csv("df.csv")

In [269]:
df = df.loc['2021-01-01':]
df.head()

,BTC-USD,^VIX,VOO,QQQ,KWEB,VNQ,TLT,BAR,COMT
date,,,,,,,,,
2021-01-01,33114.359375,33.090000,340.179993,314.559998,86.790001,84.959999,152.000000,18.309999,27.540001
2021-02-01,45137.769531,27.950001,349.589996,314.140015,89.790001,87.870003,143.119995,17.170000,30.299999
2021-03-01,58918.832031,19.400000,364.299988,319.130005,76.349998,91.860001,135.449997,16.980000,29.920000
2021-04-01,57750.175781,18.610001,383.570007,337.989990,75.139999,99.080002,138.639999,17.580000,32.130001
2021-05-01,37332.855469,16.760000,386.130005,333.929993,70.400002,99.879997,138.440002,18.920000,33.189999


In [270]:
df.replace('', np.nan, inplace=True)
df = df.dropna()

In [271]:
percent = {0:7,
           1:5,
           2:14,
           3:14,
           4:5,
           5:22,
           6:13,
           7:10,
           8:10
           }
start_summ , begin = 1000000 , 1000000
sum_per = 0
for j in range(len(percent)):
    sum_per += percent[j]
if sum_per != 100:
    print(colored(str(sum_per) + "%","red"))
    print(colored("Сумма долей не ровна 100%","yellow"))
    sys.exit()


In [272]:
rows = len(df.index)
columns = len(df.columns)

In [290]:
import datetime as dt
df['summ'] = 1000000
row_summ = 0
for i in range(rows):
    row_summ = 0
    if i > 0:
        start_summ = df.iloc[i-1:i,columns:columns+1].values
        print(pd.to_datetime(df.index[i]).strftime('%Y')) # pd.to_datetime(df['ArrivalDate']).dt.strftime('%Y-%m')

2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
